In [1]:
#ThermostatSC2

In [2]:
import datetime
import time
import random
import logging
import uuid

from utils import *

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
def get_temp_now():
    return requests.get(THERMOSTAT_API_URL + "/actual_temperature").json()


In [5]:
PREFIXES={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    }

GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                     ?meas saref:hasValue ?temp .
                     ?meas saref:isMeasuredIn saref:TemperatureUnit .
                     ?meas saref:hasTimestamp ?timestamp .
                     ?meas saref:isMeasurementOf ?room_id .
                     ?meas saref:relatesToProperty saref:Temperature .
                     ?meas saref:measurementMadeBy ?device_id ."""

In [6]:

def start_sensor_kb(kb_id, kb_name, kb_description, ke_endpoint):
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)
    ki_id = register_post_knowledge_interaction(
        GRAPH_PATTERN,
        None,
        "post-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    measurement_counter = 0
    while True:
        now = datetime.datetime.now()
        measurement_counter += 1
        value = get_temp_now()

        post(
            [
                {
                    "meas": f"<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>",
                    "temp": f"{get_temp_now()}",
                    "timestamp": f'"{get_timestamp_now()}"',  # ISO 8601 format
                    "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
                    "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
                }
            ],
            ki_id,
            kb_id,
            ke_endpoint,
        )
        logger.info(f"published measurement of {value} units at {get_timestamp_now()}")

        time.sleep(5)


In [7]:
start_sensor_kb(
    "http://example.org/sensor",
    "Thermostat1_temp",
    "A thermostat",
    "http://knowledge_engine:8280/rest/",
)

INFO:utils:registered Thermostat1_temp
INFO:utils:received issued knowledge interaction id: http://example.org/sensor/interaction/post-measurements
INFO:__main__:published measurement of 81 units at 2023-06-18T19:49:33+00:00
INFO:__main__:published measurement of 79 units at 2023-06-18T19:49:39+00:00
INFO:__main__:published measurement of 78 units at 2023-06-18T19:49:45+00:00
INFO:__main__:published measurement of 77 units at 2023-06-18T19:49:51+00:00
INFO:__main__:published measurement of 79 units at 2023-06-18T19:49:57+00:00
INFO:__main__:published measurement of 82 units at 2023-06-18T19:50:03+00:00
INFO:__main__:published measurement of 77 units at 2023-06-18T19:50:09+00:00
INFO:__main__:published measurement of 82 units at 2023-06-18T19:50:15+00:00
INFO:__main__:published measurement of 83 units at 2023-06-18T19:50:21+00:00
INFO:__main__:published measurement of 82 units at 2023-06-18T19:50:27+00:00
INFO:__main__:published measurement of 83 units at 2023-06-18T19:50:32+00:00
INFO:

KeyboardInterrupt: 